In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cmu-book-summary-dataset/booksummaries.txt


In [4]:
column_names=['Wikipedia article ID',
'Freebase ID',
'Book title',
'Author',
'Publication date',
'Book genres (Freebase ID:name tuples)',
'Plot summary']

In [5]:
data=pd.read_csv('/kaggle/input/cmu-book-summary-dataset/booksummaries.txt', delimiter='\t', names=column_names, header=None)

In [6]:
data

,Wikipedia article ID,Freebase ID,Book title,Author,Publication date,Book genres (Freebase ID:name tuples),Plot summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...
...,...,...,...,...,...,...,...
16554,36934824,/m/0m0p0hr,Under Wildwood,Colin Meloy,2012-09-25,NaN,"Prue McKeel, having rescued her brother from ..."
16555,37054020,/m/04f1nbs,Transfer of Power,Vince Flynn,2000-06-01,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction""}",The reader first meets Rapp while he is doing...
16556,37122323,/m/0n5236t,Decoded,Jay-Z,2010-11-16,"{""/m/0xdf"": ""Autobiography""}",The book follows very rough chronological ord...
16557,37132319,/m/0n4bqb1,America Again: Re-becoming The Greatness We Ne...,Stephen Colbert,2012-10-02,NaN,Colbert addresses topics including Wall Stree...


In [8]:
data['Plot summary']=data['Plot summary'].apply(lambda text:text.lower())

In [10]:
plot=data['Plot summary']

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
vectorizer=TfidfVectorizer()

In [14]:
tf_idf=vectorizer.fit_transform(plot)

In [15]:
tf_idf.shape

(16559, 121260)

In [20]:
cosine=cosine_similarity(tf_idf)

In [21]:
print(cosine)

[[1.         0.1299163  0.18956561 ... 0.14520625 0.03338112 0.11606397]
 [0.1299163  1.         0.17445624 ... 0.13893065 0.02769185 0.11881149]
 [0.18956561 0.17445624 1.         ... 0.19229554 0.0465808  0.16297015]
 ...
 [0.14520625 0.13893065 0.19229554 ... 1.         0.03152995 0.13596954]
 [0.03338112 0.02769185 0.0465808  ... 0.03152995 1.         0.02981897]
 [0.11606397 0.11881149 0.16297015 ... 0.13596954 0.02981897 1.        ]]


In [22]:
indices=pd.Series(data.index, index=data['Book title']).drop_duplicates()

In [28]:
indices

Book title
Animal Farm                                                      0
A Clockwork Orange                                               1
The Plague                                                       2
An Enquiry Concerning Human Understanding                        3
A Fire Upon the Deep                                             4
                                                             ...  
Under Wildwood                                               16554
Transfer of Power                                            16555
Decoded                                                      16556
America Again: Re-becoming The Greatness We Never Weren't    16557
Poor Folk                                                    16558
Length: 16559, dtype: int64

In [73]:
def recommend_books(title,indices,sim_matrix,number):
    if title not in indices:
        print(f"Book with name '{title}' doesn't exist")
        return []
    
    index=indices[title]
    sim_scores=list(enumerate(cosine[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    recommended=sim_scores[1:number+1]
    book_indices = [i[0] for i in recommended]
    
    return data['Book title'].iloc[book_indices].tolist()

In [78]:
# Example usage:
book_to_recommend = "A Clockwork Orange"
recommended_books = recommend_books(book_to_recommend, indices, cosine,5)

if recommended_books:
    print(f"Books similar to '{book_to_recommend}':")
    for idx, title in enumerate(recommended_books, 1):
        print(f"{idx}. {title}")
        

Books similar to 'A Clockwork Orange':
1. Little Boy Blue
2. Point Blanc
3. Scorpia Rising
4. Point Blanc: The Graphic Novel
5. Snakehead
